# Лабораторная работа по Теории кодирования
## Лабораторная работа №6
Выполнили студенты группы 6401-010302D

Баловнева Юлия

Богатырев Дмитрий


In [39]:
from ast import Bytes
from venv import create
from xmlrpc.client import Boolean

import numpy as np
import itertools

from numpy import dtype
from numpy.f2py.auxfuncs import throw_error
from numpy.ma.core import zeros


def mult(m1, m2):
    '''Умножает две матрицы xor-ом и возвращает результат
    :param:
        m1: первая матрица
        m2: вторая матрица
        
        :return: C: возвращает матрицу - xor-произведение двух матриц 
    '''
    A = np.array(m1, dtype=int)
    B = np.array(m2, dtype=int)
    C = (A@B)%2
    return np.array(C, dtype=bool)

def code_length(matr):
    '''Вычисляет кодовое расстояние массива переданных слов
    :param:
        matr: матрица - двумерный bool-массив - масиив двоичных слов
        
        :return: d: кодовое расстояние. 
    '''
    d = len(matr[0])
    for i in range(len(matr)-1):
        for j in range(i+1,len(matr)):
            # Ищем минимальное число единиц в множестве произведений i и j строк
            d = min(d,sum(np.bitwise_xor(matr[i],matr[j])))
    return d

def create_verif_matr(X):
    k = X.shape[1]
    P = np.eye(k,dtype=bool)
    return np.vstack([P,X])

def create_rref_matr(X):
    k = X.shape[0]
    P = np.eye(k,dtype=bool)
    return np.hstack([P,X])

def kronekers_mult(A,B):
    i_stack = None
    for i in range(A.shape[0]):
        j_stack = None
        for j in range(A.shape[1]):
            aijB = A[i,j]*B
            if j_stack is None:
                j_stack = aijB
            else:
                j_stack = np.hstack((j_stack,aijB))
                
        if i_stack is None:
            i_stack = j_stack
        else:
            i_stack = np.vstack((i_stack,j_stack))
            
    return i_stack

def inner_shape(matr, dimension):
    ndims = matr.ndim
    if ndims == 1 and dimension == 1:
        return matr.shape[0]
    elif ndims == 1 and dimension == 0:
        return 1
    elif ndims == 2:
        return matr.shape[dimension]
    else:
        print("Unidefined dimension error")
        return -1
    
def to_binary_np(n,m):
    return np.array(list(reversed(list(np.binary_repr(n, width=m)))), dtype=int)


In [40]:
class LinearCode:
    ''' Класс линейных кодов.

    Атрибуты:\n
    matr: копия исходной матрицы\n
    k: количество строк матрицы\n
    n: количество столбцов матрицы
    
    '''
    # Конструктор
    def __init__(self, n_array):
        self.matr = np.array(n_array, dtype=bool)
        self.n = n_array.shape[1]
        self.k = n_array.shape[0]
    
    def find_first(self, line):
        '''Поиск первого ненулевого элемента в строке.
        
        :param:
        line: строка матрицы.
        
        :return: h: возвращаем позицию первого ненулевого элемента в строке. 
        '''
        # Задаем h = количеству элементов в сторке
        h = self.n
        # В строке line перебираем элементы пока не натолкнемся на 1
        for i in range(self.n):
            if self.matr[line,i]:
                h = i
                break
        # Возвращаем позицию найденной единицы в строке
        return h
    
    def xor_swap(self, line1, line2):
        '''Меняет две строки местами с помощью поэлементного XOR.

        :param:
            line1: первая строка.\n
            line2: вторая строка.

        '''
        # Поэлементный XOR 
        for w in range(self.n):
            self.matr[line1,w] ^= self.matr[line2,w]
            self.matr[line2,w] ^= self.matr[line1,w]
            self.matr[line1,w] ^= self.matr[line2,w]


    def sum(self, line1, line2):
        '''Складывает первую переданную строку со второй.

        :param:
            line1: первая строка.\n
            line2: вторая строка.
        '''
        # Поэлементный XOR 
        for w in range(self.n):
            self.matr[line2,w] ^= self.matr[line1,w]

    def __str__(self):
        '''Собирает строку (смена типа от bool к string).

        :return: s: строка с матрицей.            
        '''
        s = ''
        for i in range(self.k):
            s += '['
            for j in range(self.n):
                if self.matr[i,j]: s += "1 "
                else: s += "0 "
            s += "\b]\n"
        return s
    
    def fillrand(self):
        '''Заполняет матрицу случайным образом нулями и единицами.
        
        '''
        rand = np.random
        for i in range(self.k):
            for j in range(self.n):
                self.matr[i,j] = np.round(rand.random())

    def sort(self):
        '''Сортировка строк матрицы.
        '''
        # sort
        for i in range(self.k):
            for j in range(self.k-i-1):
                if self.find_first(j) > self.find_first(j+1):
                    self.xor_swap(j, j+1)

    def ref(self):
        '''Приводит матрицу к ступенчатому виду.
        
            ---
            Ступенчатая матрица - такая матрица, что
            * все ненулевые строки располагаются над всеми чисто нулевыми строками
            * ведущий элемент (первый, считая слева направо, ненулевой элемент строки)
            каждой ненулевой строки располагается строго правее ведущего элемента в
            строке, расположенной выше данной.
            ---
        '''
        # Проходит по всем строкм матрицы
        for i in range(self.k-1):
            # Сравнивает рабочую строку с текущей
            for j in range(i+1, self.k):
                # Запоминаем первые элементы
                first_i = self.find_first(i)
                first_j = self.find_first(j)
                # Если они равны, то делаем xor со строкой
                # Поскольку рабочая строка всегда ниже текущей, то при xor первый элемент всех нижних строк зануляется
                if first_i == first_j:
                    self.sum(i,j)
                # Если встретили строку, у которой первый элемент раньше чем у текущей, то меняем строки местами
                elif first_i > first_j:
                    self.xor_swap(i,j)
                # Иначе все хорошо, идем дальше
        # Удаляем занулившиеся строки, которые могли бы получится
        self.delete_redundant_lines()
        

    def rref(self):
        '''Приводит матрицу к приведенному ступенчатому виду. 
        
            ---
            Ступенчатая матрица называется приведенной, если матрица,  не имеет нулевых строк, 
            и все ведущие элементы ее строк равны единице. При этом все элементы основных столбцов, 
            помимо ведущих элементов, являются нулями.
            
        '''
        # Сначала приводим матрицу к ступенчатому виду
        self.ref()
        # Дальше для всех строк...
        for i in range(1,self.k):
            # ...мы ищем номер ведущего элемента строки...
            ist = self.find_first(i)
            # ...и делаем xor для всех строк выше текущей
            for j in range(i-1,-1,-1):
                if(self.matr[j,ist]):
                    self.sum(i,j)
            # Таким образом зануляются все элементы над ведущим
        # Удаляем занулившиеся строки, которые могли бы получится
        self.delete_redundant_lines()
    

    def delete_redundant_lines(self):
        '''Удаляет нулевые строки из матрицы.
        '''
        # Копия матрицы, с которой ведется работа
        temp = self.matr.copy()
        offset = 0 # сдвиг
        for i in range(self.k):
            # Если номер первой единицы в строке == кол-ву элементов самой строки (то есть единиц там нет)
            # Удаляем эту строку и сдвигаем указатель на 1
            if self.find_first(i) == self.n:
                temp = np.delete(temp, i - offset, 0)
                offset += 1
        # Перезаписываем матрицу на новую и очищенную
        self.matr = temp
        # Обновляем переменные-размеры матрицы
        self.n = temp.shape[1]
        self.k = temp.shape[0]

    
    def create_reduced_matr(self):
        rrefed_matr = LinearCode(self.matr.copy())
        rrefed_matr.rref()

        # Определяем индексы ведущих элементов
        leads = np.array([rrefed_matr.find_first(i) for i in range(rrefed_matr.k)])

        # Создаем сокращенную матрицу 
        X = np.zeros(shape=(rrefed_matr.k,rrefed_matr.n-rrefed_matr.k), dtype=bool)
        
        # Заполняем ее значениями из ступенчатой
        for i in range(rrefed_matr.k):
            offset = 0 # сдвиг
            for j in range(rrefed_matr.n):
                # Если в столбце содержится ведущий элемент, то его пропускаем (и увеличиваем сдвиг на 1)
                # Иначе добавляем этот элемент в сокращенную матрицу
                if j in leads: 
                    offset+=1
                else:
                    X[i,j-offset] = rrefed_matr.matr[i,j]
        return LinearCode(X)

                
    def create_verif_matr(self):
        '''Создает проверочную матрицу на основе порождающей.
        
        :return: LinearCode(H): проверочная матрица. 
        '''
        # Создаем новую матрицу в приведенном ступенчатом виде на основе текущей
        rrefed_matr = LinearCode(self.matr.copy())
        rrefed_matr.rref()

        # Определяем индексы ведущих элементов
        leads = np.array([rrefed_matr.find_first(i) for i in range(rrefed_matr.k)])

        # Создаем сокращенную матрицу 
        X = np.zeros(shape=(rrefed_matr.k,rrefed_matr.n-rrefed_matr.k), dtype=bool)
        
        # Заполняем ее значениями из ступенчатой
        for i in range(rrefed_matr.k):
            offset = 0 # сдвиг
            for j in range(rrefed_matr.n):
                # Если в столбце содержится ведущий элемент, то его пропускаем (и увеличиваем сдвиг на 1)
                # Иначе добавляем этот элемент в сокращенную матрицу
                if j in leads: 
                    offset+=1
                else:
                    X[i,j-offset] = rrefed_matr.matr[i,j]

        # Единичная матрица
        Ind = np.identity(n = rrefed_matr.n-rrefed_matr.k, dtype = bool)

        # Создаем проверочную матрицу
        H = np.zeros(shape=(rrefed_matr.n,rrefed_matr.n-rrefed_matr.k), dtype=bool)
        # сдвиги
        offset_i = 0
        offset_matr = 0
        # Заполняем ее значениями
        for i in range(H.shape[0]):
            # Если номер строки содержится в массиве с ведущими элементами, то заполняем строку из сокращенной матрицы
            # И прибавляем 1 к сдвигу для элементов единичной матрицы
            if i in leads:
                for j in range(H.shape[1]):
                    H[i,j] = X[i-offset_matr,j]
                offset_i += 1
            # Иначе заполняем строку из единичной матрицы
            # И прибавляем 1 к сдвику для матрицы сокращенной
            else:
                for j in range(H.shape[1]):
                    H[i,j] = Ind[i-offset_i,j]
                offset_matr += 1
        
        return LinearCode(H)
    
    def gen_codewords_summing(self):
        ''' Формирует набор слов путем сложения всех слов из порождающего множества

        :return: wordset: набор кодовых слов
        '''
        # Создаем пустой набор слов и доюавляем в него 0й элемент
        wordset = set()
        wordset.add(tuple((np.zeros(self.n, dtype=bool))))
        # Для i числа строк
        for i in range(1, self.k+1):
            # Массив длинны i всех возможных комбинаций строк 
            combinations = np.array(list(itertools.combinations(range(self.k), i)))
            for comb in combinations:
                # Для каждой комбинации
                # Создаем пустое слово
                word = np.zeros(self.n, dtype=bool)
                # Записываем в него сумму всех строк в текущей комбинации строк
                for j in comb:
                    word ^= self.matr[j]
                # Добавляем слово в набор слов
                wordset.add(tuple(word.tolist()))
        return wordset
    
    def gen_codewords_bin(self):
        ''' Формирует набор слов путем умножения всех двоичных слов длины k на G 

        :return: wordset: набор кодовых слов
        '''
        # Копия текущей матрицы
        G = LinearCode(self.matr)
        # Приводим в ступенчатому виду
        G.ref
        # Пустой набор слов
        wordset = set()
        # Набор всех возможных двоичных слов длины k
        binset = set(tuple(itertools.product((True,False),repeat=self.k)))
        # Для каждой комбинации
        for bin in binset:
            # Умножаем комбинацию на ступенчатую матрицу
            # И записываем слово в набор
            word = tuple((mult(bin,G.matr)).tolist())
            wordset.add(word)
        return wordset



#### 6.1

#### $$g(x)=1+x^2+x^3$$

In [41]:
def cyclic_shift(v):
    w = v.copy()
    x = w.pop(-1)
    w.insert(0,x)
    return w

def create_G_for_poly(g,n,k):
    G = list()
    line = list(g) + list(np.zeros((n-len(g)),dtype=int))
    for i in range(k):
        G.append(line)
        line=cyclic_shift(line)
    return np.array(G)

create_G_for_poly([1,0,1,1],7,4)
        

array([[1, 0, 1, 1, 0, 0, 0],
       [0, 1, 0, 1, 1, 0, 0],
       [0, 0, 1, 0, 1, 1, 0],
       [0, 0, 0, 1, 0, 1, 1]])

In [93]:
def find_last(v):
    for i in reversed(range(len(v))):
        if v[i] == 1:
            return i
    return 0

def extend_word(w,n):
    if(len(w)<n):
        w = w + [0 for i in range(n-len(w))]
    return w

def poly_div(p,q):
    if len(p)!= len(q):
        print("Different lengths!")
        return None
    
    n = find_last(p)
    k = find_last(q)
    p_in_div = p.copy()
    div_poly = [0 for i in range(n-k+1)]

    while n>=k:
        q_in_div = q
        for i in range(n-k):
            q_in_div = cyclic_shift(q_in_div)
        p_in_div = [p_in_div[i] ^ q_in_div[i] for i in range(len(p))]
        div_poly[n-k] += 1
        n = find_last(p_in_div)
    return div_poly, p_in_div

print(poly_div([0,1,1,0,0,0,1,1,1],[1,1,1,0,1,0,0,0,0]))
    

([0, 0, 0, 1, 1], [0, 1, 1, 1, 0, 0, 0, 0, 0])


In [87]:
def check_poly(w,g,t):
    s = poly_div(w,g)[1]
    s_mod = poly_div(s,g)[1]
    
    for i in range(len(s)):
        si = s_mod.copy()
        for j in range(i):
            si = cyclic_shift(si)
        if np.sum(si)<=t:
            e = si.copy()
            for j in range(len(w)-i):
                e = cyclic_shift(e)
            return e
    print("Unable to correct the error")
    return None

In [94]:
v = [0,1,0,1]
g = [1,0,1,1,0,0,0]
w = mult(v,create_G_for_poly(g,7,4))

print("Исходное слово: ")
print(v) 


print("Закодированное слово: ")
print(w) 

print("Синдром: ")
syndrome = check_poly(w,g,1)
print(syndrome)

print("Декодированное слово: ")
w_decoded = extend_word(poly_div([w[i] ^ syndrome[i] for i in range(len(w))],g)[0],4)
print(w_decoded)

print("Совпадают ли?")
print(v==w_decoded) 
# Совпадают

Исходное слово: 
[0, 1, 0, 1]
Закодированное слово: 
[False  True False False  True  True  True]
Синдром: 
[np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0)]
Декодированное слово: 
[0, 1, 0, 1]
Совпадают ли?
True


In [95]:
v = [0,1,0,1]
g = [1,0,1,1,0,0,0]
w = mult(v,create_G_for_poly(g,7,4))
errors = np.eye(7,dtype=bool)
print("Исходное слово: ")
print(v) 


print("Закодированное слово: ")
print(w) 

w_err = w ^ errors[1]
print("Закодированное слово с ошибкой: ")
print(w_err) 

print("Синдром: ")
syndrome = check_poly(w_err,g,1)
print(syndrome)

print("Декодированное слово: ")
w_decoded = extend_word(poly_div([w[i] ^ syndrome[i] for i in range(len(w))],g)[0],4)
print(w_decoded)

print("Совпадают ли?")
print(v==w_decoded) 
# Ошибка обнаружена и исправлена

Исходное слово: 
[0, 1, 0, 1]
Закодированное слово: 
[False  True False False  True  True  True]
Закодированное слово с ошибкой: 
[False False False False  True  True  True]
Синдром: 
[np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0)]
Декодированное слово: 
[0, 1, 0, 1]
Совпадают ли?
True


In [96]:
print("Исходное слово: ")
print(v) 


print("Закодированное слово: ")
print(w) 

w_err = w ^ errors[1] ^ errors[2]
print("Закодированное слово с ошибкой: ")
print(w_err) 

print("Синдром: ")
syndrome = check_poly(w_err,g,1)
print(syndrome)

print("Декодированное слово: ")
w_decoded = extend_word(poly_div([w[i] ^ syndrome[i] for i in range(len(w))],g)[0],4)
print(w_decoded)

print("Совпадают ли?")
print(v==w_decoded) 
# Ошибка обнаружена, но не исправлена

Исходное слово: 
[0, 1, 0, 1]
Закодированное слово: 
[False  True False False  True  True  True]
Закодированное слово с ошибкой: 
[False False  True False  True  True  True]
Синдром: 
Unable to correct the error
None
Декодированное слово: 


TypeError: 'NoneType' object is not subscriptable

In [97]:
print("Исходное слово: ")
print(v) 


print("Закодированное слово: ")
print(w) 

w_err = w ^ errors[1] ^ errors[2] ^ errors[3]
print("Закодированное слово с ошибкой: ")
print(w_err) 

print("Синдром: ")
syndrome = check_poly(w_err,g,1)
print(syndrome)

print("Декодированное слово: ")
w_decoded = extend_word(poly_div([w[i] ^ syndrome[i] for i in range(len(w))],g)[0],4)
print(w_decoded)

print("Совпадают ли?")
print(v==w_decoded) 
# Ошибка обнаружена, но не исправлена

Исходное слово: 
[0, 1, 0, 1]
Закодированное слово: 
[False  True False False  True  True  True]
Закодированное слово с ошибкой: 
[False False  True  True  True  True  True]
Синдром: 
Unable to correct the error
None
Декодированное слово: 


TypeError: 'NoneType' object is not subscriptable

#### 6.2
$$g(x)=1+x^3+x^4+x^5+x^6$$

In [143]:
def check_poly_t(w,g,t):
    def error_set(t,n):
        packet_list = [list(to_binary_np(i+1,n)) for i in range(pow(2,t))]
        error_set = set()
        for i in range(n):
            for j in range(len(packet_list)):
                error_set.add(tuple(packet_list[j]))
                packet_list[j] = cyclic_shift(packet_list[j])
        return error_set
    
    s = poly_div(w,g)[1]
    s_mod = poly_div(s,g)[1]
    for i in range(len(s)):
        si = s_mod.copy()
        for j in range(i):
            si = cyclic_shift(si)
        if tuple(si) in error_set(t,len(w)):
            e = si.copy()
            for j in range(len(w)-i):
                e = cyclic_shift(e)
            return e
    print("Unable to correct the error")
    return None

In [144]:
v = [0,1,0,1,1,0,0,0,0]
g = [1,0,0,1,1,1,1,0,0,0,0,0,0,0,0]
w = mult(v,create_G_for_poly(g,15,9))
errors = np.eye(15,dtype=bool)
print("Исходное слово: ")
print(v) 


print("Закодированное слово: ")
print(w) 

w_err = w ^ errors[1]
print("Закодированное слово с ошибкой: ")
print(w_err) 

print("Синдром: ")
syndrome = check_poly_t(w_err,g,3)
print(syndrome)

print("Декодированное слово: ")
w_decoded = extend_word(poly_div([w[i] ^ syndrome[i] for i in range(len(w))],g)[0],9)
print(w_decoded)

print("Совпадают ли?")
print(v==w_decoded) 
# Ошибка обнаружена и исправлена

Исходное слово: 
[0, 1, 0, 1, 1, 0, 0, 0, 0]
Закодированное слово: 
[False  True False  True False  True False  True False False  True False
 False False False]
Закодированное слово с ошибкой: 
[False False False  True False  True False  True False False  True False
 False False False]
Синдром: 
[np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0)]
Декодированное слово: 
[0, 1, 0, 1, 1, 0, 0, 0, 0]
Совпадают ли?
True


In [145]:
print("Исходное слово: ")
print(v) 


print("Закодированное слово: ")
print(w) 

w_err = w ^ errors[1] ^ errors[2]
print("Закодированное слово с ошибкой: ")
print(w_err) 

print("Синдром: ")
syndrome = check_poly_t(w_err,g,3)
print(syndrome)

print("Декодированное слово: ")
w_decoded = extend_word(poly_div([w[i] ^ syndrome[i] for i in range(len(w))],g)[0],9)
print(w_decoded)

print("Совпадают ли?")
print(v==w_decoded) 
# Ошибка обнаружена и исправлена

Исходное слово: 
[0, 1, 0, 1, 1, 0, 0, 0, 0]
Закодированное слово: 
[False  True False  True False  True False  True False False  True False
 False False False]
Закодированное слово с ошибкой: 
[False False  True  True False  True False  True False False  True False
 False False False]
Синдром: 
[np.int64(0), np.int64(1), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0)]
Декодированное слово: 
[0, 1, 0, 1, 1, 0, 0, 0, 0]
Совпадают ли?
True


In [147]:
print("Исходное слово: ")
print(v) 


print("Закодированное слово: ")
print(w) 

w_err = w ^ errors[2] ^ errors[4]
print("Закодированное слово с ошибкой: ")
print(w_err) 

print("Синдром: ")
syndrome = check_poly_t(w_err,g,3)
print(syndrome)

print("Декодированное слово: ")
w_decoded = extend_word(poly_div([w[i] ^ syndrome[i] for i in range(len(w))],g)[0],9)
print(w_decoded)

print("Совпадают ли?")
print(v==w_decoded) 
# Ошибка обнаружена и исправлена

Исходное слово: 
[0, 1, 0, 1, 1, 0, 0, 0, 0]
Закодированное слово: 
[False  True False  True False  True False  True False False  True False
 False False False]
Закодированное слово с ошибкой: 
[False  True  True  True  True  True False  True False False  True False
 False False False]
Синдром: 
[np.int64(0), np.int64(0), np.int64(1), np.int64(0), np.int64(1), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0)]
Декодированное слово: 
[0, 1, 0, 1, 1, 0, 0, 0, 0]
Совпадают ли?
True


In [148]:
print("Исходное слово: ")
print(v) 


print("Закодированное слово: ")
print(w) 

w_err = w ^ errors[1] ^ errors[2]^ errors[4]
print("Закодированное слово с ошибкой: ")
print(w_err) 

print("Синдром: ")
syndrome = check_poly_t(w_err,g,3)
print(syndrome)

print("Декодированное слово: ")
w_decoded = extend_word(poly_div([w[i] ^ syndrome[i] for i in range(len(w))],g)[0],9)
print(w_decoded)

print("Совпадают ли?")
print(v==w_decoded) 
# Ошибка обнаружена но не исправлена

Исходное слово: 
[0, 1, 0, 1, 1, 0, 0, 0, 0]
Закодированное слово: 
[False  True False  True False  True False  True False False  True False
 False False False]
Закодированное слово с ошибкой: 
[False False  True  True  True  True False  True False False  True False
 False False False]
Синдром: 
Unable to correct the error
None
Декодированное слово: 


TypeError: 'NoneType' object is not subscriptable